In [1]:
import networkx as nx
import os
import json
import pathlib
import tqdm

In [2]:
def _init_graph(G, growth_steps, built=True):
    if built:
        init_edges = [edge for edge in G.edges if G.edges[edge]["built"] == 1]
    # Find initial edges if not built by finding the ones that are not on the growth steps. Supposedly it's a single random edge from the highest closeness node, see growth.order_network_growth
    else:
        init_edges = [edge for edge in G.edges if edge not in growth_steps]
    return G.edge_subgraph(init_edges)

In [3]:
for graphname in tqdm.tqdm(
    ["radio_concentric", "grid", "grid_with_diagonal", "three_bridges"]
):
    print(f"{graphname}")
    folderoots = f"../data/processed/ignored_files/paper/{graphname}/"
    if not os.path.exists(folderoots + "plots/"):
        os.makedirs(folderoots + "plots/")
    # Open for all toy graphs
    toy_graph_aucs = []
    # Open for all growth strategies for a toy graph
    G = nx.read_graphml(folderoots + "graph.graphml")
    for growth_f in [
        fold
        for fold in pathlib.Path(folderoots).iterdir()
        if not fold.is_file()
        and "plot" not in str(fold)
        and ".DS_Store" not in str(fold)
        and "manual" not in str(fold)
    ]:
        growthname = str(growth_f).split("/")[-1].removesuffix("_connected")
        if "additive" in growthname:
            order = "additive"
        elif "subtractive" in growthname:
            order = "subtractive"
        met_name = "_".join(growthname.split("_")[:-1])
        for trial in sorted(
            [fold for fold in pathlib.Path(growth_f).iterdir() if "order" in str(fold)]
        ):
            with open(trial, "r") as f:
                order_dict = json.load(f)
            order_dict = [tuple(val) for val in order_dict]
            G_init = _init_graph(G, order_dict, built=False)
            actual_edges = list(G_init.edges)
            for edge in order_dict:
                actual_edges.append(edge)
                G_actual = G.edge_subgraph(actual_edges)
                if nx.number_connected_components(G_actual) > 1:
                    print(f"Issue with {trial}")

  0%|          | 0/4 [00:00<?, ?it/s]

radio_concentric


 25%|██▌       | 1/4 [00:43<02:11, 43.93s/it]

grid


 50%|█████     | 2/4 [05:05<05:43, 171.74s/it]

grid_with_diagonal


 75%|███████▌  | 3/4 [07:04<02:27, 147.85s/it]

three_bridges


100%|██████████| 4/4 [09:05<00:00, 136.32s/it]
